In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
import zipfile
from typing import Optional, Union, List, Callable, Dict, Tuple
from dataclasses import dataclass, make_dataclass, field
from abc import ABC
from importnb import Notebook

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [ ]:
with Notebook():
    from src.basestructs import *
    from src.baseblocks import *

In [10]:
class Globals(RecordsBlock):
    
    Pattern = RecordBase(segList=[('NAME', 32, str)
                                 ,('TYPE', 2, int)
                                 ,('REF', 2, int)
                                 ,('DWORD', 4, int)
                                 ,('INT', 4, int)
                                 ,('DOUBEL', 8, int)
                                 ,('SCRIPT', 32, str)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


In [11]:
class KnownSpells(RecordsBlock):
    
    Pattern = RecordBase(segList=[('SPELL', 8, 'STRNAME')
                                 ,('LEVEL', 2, int)
                                 ,('TYPE', 2, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.SPELL
        
    def post_op(self):
        self.df['LEVEL'] = self.Pattern.inverse_infer_col((self.Pattern.infer_col(self.df['LEVEL']) - 1).clip(0))
        super().post_op()


In [12]:
class SpellMemorization(RecordsBlock):
    
    Pattern = RecordBase(segList=[('LEVEL', 2, int)
                                 ,('BASECOUNT', 2, int)
                                 ,('EFFCOUNT', 2, int)
                                 ,('TYPE', 2, int)
                                 ,('INDEX', 4, int)
                                 ,('COUNTMEM', 4, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
            
    def post_op(self):
        self.df['LEVEL'] = self.Pattern.inverse_infer_col((self.Pattern.infer_col(self.df['LEVEL']) - 1).clip(0))
        self.df.sort_values(['TYPE', 'LEVEL'], inplace=True)
        super().post_op()
        
    def update_mem(self, idf:pd.DataFrame):
        '''
        Update SpellMeorization info using he memorized spells. All memorization slot numbers matching the number of memorized spells (c.a. assuming all slots memorized)
        '''
        idf = self.Pattern.inverse_infer(idf)
        self.df = pd.merge(self.df.drop(['COUNTMEM', 'INDEX'], axis=1), idf, how='left', left_on=['LEVEL', 'TYPE'], right_on=['LEVEL', 'TYPE'])
        self.df.fillna(value={'INDEX': self.df['INDEX'].ffill().bfill()
                              ,'COUNTMEM': b'\x00\x00\x00\x00'}, inplace=True)
        self.df['BASECOUNT'] = self.df['COUNTMEM']
        self.df['EFFCOUNT'] = self.df['COUNTMEM']


In [13]:
class MemorizedSpells(RecordsBlock):
    
    TypeMap = {b'SPWI': 1
              ,b'SPCL': 3
              ,b'SPIN': 2
              ,b'SPPR': 0
              ,b'SPSD': 4}
    
    Pattern = RecordBase(segList=[('SPELL', 8,'STRNAME')
                                 ,('COUNTMEM', 4, int)])
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.SPELL
        
    def post_op(self):
        self.df.insert(0, 'TYPE', self.df['SPELL'].apply(lambda x: self.TypeMap.get(x[:4])))
        self.df.insert(0, 'LEVEL', self.df['SPELL'].apply(lambda x: int(x[4:5])))
        self.df = self.df.sort_values(['TYPE', 'LEVEL'])
        idf = self.df.groupby(['TYPE', 'LEVEL'])['COUNTMEM'].count().reset_index()
        idf.insert(idf.shape[1] - 1, 'INDEX', idf['COUNTMEM'].shift(fill_value=0).cumsum())
        self.df = self.df.sort_values(['TYPE', 'LEVEL', 'SPELL']).drop(['TYPE', 'LEVEL'], axis=1).reset_index(drop=True)
        self.parentRef.SpellMemorization.update_mem(idf)
        super().post_op()


In [14]:
class Effects(RecordsBlock):
    Pattern = RecordBase(segList=[('NULL', 8, int)
                                 ,('OPCODE', 4, 'INTNAME')
                                 ,('TARGET', 4, int)
                                 ,('POWER', 4, int)
                                 ,('PARAM1', 4, int)
                                 ,('PARAM2', 4, int)
                                 ,('TIME', 4, int)
                                 ,('OTHER', 232, None)]
                         ,template=bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                              0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                              255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                              0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.EFFECT


In [15]:
class Items(RecordsBlock):
    
    Pattern = RecordBase(segList=[('NAME', 8, 'STRNAME')
                                 ,('EXPIRE', 1, int)
                                 ,('ELAPSED', 1, int)
                                 ,('QUALITY1', 2, int)
                                 ,('QUALITY2', 2, int)
                                 ,('QUALITY3', 2, int)
                                 ,('IDENTIFIED', 1, int)
                                 ,('UNSTEALABLE', 1, int)
                                 ,('STOLEN', 1, int)
                                 ,('UNDROPPABLE', 1, int)]
                         ,template=b'CLCK30\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x01\x00\x00\x00')
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.Pattern.nameMap = self.savRef.ITEM
        
    def post_op(self):
        super().post_op()
        self.parentRef.ItemSlots.place(self.display['NAME'])

In [16]:
class ItemSlots(SegBlock):
    
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.slots = {k: v.value.copy() for k, v in getattr(self.savRef, self.parentRef.parentRef.name.upper() + 'SLOTS').__members__.items()}
        
    def place(self, itms:pd.Series):
        fill = [-1] * (max([max(_) for _ in self.slots.values()]) + 1)
        bins = [_ for _ in self.slots.keys() if _ != 'NONE']
        for i, itm in enumerate(itms):
            slot = getattr(self.savRef.ITEMCONTAINER, itm).value
            for cname in [slot, 'MISC', *bins]:
                c = self.slots.get(cname)
                if c is not None and len(c) > 0:
                    fill[c.pop(0)] = i
                    break
        self.buffer = bytearray(b''.join([self.num2Bytes(_, length=2, signed=True) for _ in fill]))
        
            